In [52]:
import pandas as pd
import numpy as np
import warnings
from joblib import Parallel,delayed
from dateutil.relativedelta import relativedelta
warnings.filterwarnings('ignore')

In [53]:
#波动率因子

In [54]:
daily_return=pd.read_csv('D:/causis/daily_return.csv',parse_dates=['Clock'],index_col=0)

In [55]:
#构建因子接收池，时间为2011至2020年
pool=pd.DataFrame(index=daily_return.index,columns=daily_return.columns)
pool=pool.replace(np.nan,0).resample(rule="m").mean()
pool=pool.replace(0,np.nan)
pool=pool["20110101":"20201231"]

In [59]:
#修改condition函数可获取不同的因子，这里获取的是波动率因子，通过此框架获得不同时间窗口计算的波动率因子
#对历史收益率求标准差
def condition_std(t):
    if t.isna().sum(axis=0)<=84:  #此处作用为将计算窗口不足2/3的值去除
        std=t.std()
        return std
    else:
        pass
#使用并行计算加快速度
def apply_parallel(lm,func):
    results = Parallel(n_jobs=7,backend='loky',verbose=True)(delayed(func)(t) for t in range(len(lm)))
    return results
#定义滚动窗口，对窗口中的数据使用condition_std
def get_std_result(i):
    starttime=pool.index[i]-relativedelta(months=12)  #此处为滚动窗口，可设置滚动窗口参数
    part_ret=daily_return[starttime:pool.index[i]]  #截取计算窗口所需要的值
    part_std=part_ret.apply(condition_std)
    return part_std

In [5]:
std_12m=apply_parallel(pool.index,get_std_result)

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    4.2s
[Parallel(n_jobs=7)]: Done 120 out of 120 | elapsed:   10.9s finished


In [6]:
for k in range(len(std_12m)):
    pool.iloc[k]=std_12m[k]
pool

,S.CN.SSE.600000,S.CN.SSE.600004,S.CN.SSE.600006,S.CN.SSE.600007,S.CN.SSE.600008,S.CN.SSE.600009,S.CN.SSE.600010,S.CN.SSE.600011,S.CN.SSE.600012,S.CN.SSE.600015,...,S.CN.SZSE.300805,S.CN.SZSE.300806,S.CN.SZSE.300807,S.CN.SZSE.300808,S.CN.SZSE.300809,S.CN.SZSE.300810,S.CN.SZSE.300811,S.CN.SZSE.300812,S.CN.SZSE.300813,S.CN.SZSE.300815
Clock,,,,,,,,,,,,,,,,,,,,,
2011-01-31,0.019086,0.018978,0.020929,0.022385,0.025025,0.022910,0.023448,0.014846,0.022377,0.021643,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-02-28,0.018710,0.018640,0.020984,0.022975,0.022602,0.022548,0.024545,0.014703,0.022030,0.021691,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-03-31,0.018544,0.018179,0.020684,0.022690,0.022410,0.022289,0.029167,0.014513,0.021864,0.021282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-04-30,0.018281,0.017542,0.020005,0.023375,0.022024,0.022450,0.031225,0.015719,0.021097,0.021780,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-05-31,0.017163,0.016234,0.018276,0.021699,0.021105,0.020321,0.032715,0.016371,0.019945,0.019755,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,0.014418,0.025612,0.021642,0.019022,0.015335,0.020433,0.016277,0.018965,0.014347,0.011572,...,0.048411,0.046626,0.040333,0.036680,0.026012,0.044123,0.041702,NaN,NaN,NaN
2020-09-30,0.014280,0.024889,0.021556,0.019246,0.015416,0.020819,0.017946,0.020120,0.014323,0.011501,...,0.047722,0.045811,0.039672,0.036117,0.026896,0.042754,0.040171,0.045118,0.046495,0.041959
2020-10-31,0.013706,0.023966,0.022835,0.019055,0.015465,0.020278,0.018059,0.020666,0.014609,0.011543,...,0.050445,0.044982,0.042387,0.036204,0.026920,0.042348,0.039984,0.044102,0.046089,0.041050


In [7]:
pool.to_csv("D:/std_12m.csv")

In [ ]:
#估值因子

In [2]:
valuation_data=pd.read_csv('D:/causis/valuation_data.csv')

In [3]:
valuation_data=valuation_data.set_index("Date")
valuation_data.index=pd.to_datetime(valuation_data.index)

In [7]:
#使用数据透析表函数，得到index为时间，columns为股票代码的dataframe
EP=1/valuation_data.pivot_table(index=["Date"],columns=["Symbol"],values="PeRatio")
BP=1/valuation_data.pivot_table(index=["Date"],columns=["Symbol"],values="PBRatio")
SP=1/valuation_data.pivot_table(index=["Date"],columns=["Symbol"],values="PSRatio")
NCFP=1/valuation_data.pivot_table(index=["Date"],columns=["Symbol"],values="PCFRatio")

In [37]:
#取每个因子的月末值
valuation_factorlist=[EP,BP,SP,NCFP]
for i in range(len(valuation_factorlist)):
    valuation_factorlist[i]=valuation_factorlist[i].groupby([valuation_factorlist[i].index.year,valuation_factorlist[i].index.month]).tail(1) #取因子每月末的取值

In [20]:
valuation_factorlist[0].to_csv("D:/EP.csv")
valuation_factorlist[1].to_csv("D:/BP.csv")
valuation_factorlist[2].to_csv("D:/SP.csv")
valuation_factorlist[3].to_csv("D:/NCFP.csv")

In [ ]:
#财务质量因子

In [21]:
finance_data=pd.read_csv('D:/causis/finance_data.csv')

In [23]:
finance_data=finance_data.set_index("StatDate")
finance_data.index=pd.to_datetime(finance_data.index)

In [44]:
finance_factor_name=["ROE","ROA","GrossProfitMargin","AdjustedProfitToProfit","IncRevenueYearOnYear","IncNetProfitYearOnYear"]
finance_factorlist=[]
for t in finance_factor_name:
    part_factor=finance_data.pivot_table(index=["StatDate"],columns=["Symbol"],values=t) #取出财务因子相应的dataframe，index为时间，columns为股票代码
    part_factor=pd.DataFrame(part_factor,index=valuation_factorlist[0].index).fillna(method="ffill")#由于因子为月频，而财务数据每季度才有值，这里使用fillna函数填充空缺值
    finance_factorlist.append(part_factor)

In [46]:
finance_factorlist[0].to_csv("D:/ROE.csv")
finance_factorlist[1].to_csv("D:/ROA.csv")
finance_factorlist[2].to_csv("D:/GrossProfitMargin.csv")
finance_factorlist[3].to_csv("D:/AdjustedProfitToProfit.csv")
finance_factorlist[4].to_csv("D:/IncRevenueYearOnYear.csv")
finance_factorlist[5].to_csv("D:/IncNetProfitYearOnYear.csv")

In [ ]:
#换手率因子

In [49]:
turnover=valuation_data.pivot_table(index=["Date"],columns=["Symbol"],values="TurnoverRatio")/100

In [80]:
pool1=pd.DataFrame(index=pool.index,columns=pool.columns)

In [81]:
#condition_turnover为计算历史换手率均值
def condition_turnover(t):
    if t.isna().sum(axis=0)<=84:  #此处作用为将计算窗口不足2/3的值去除
        mean=t.mean()
        return mean
    else:
        pass
#定义滚动窗口，对窗口中的数据使用condition_turnover
def get_turnover_result(i):
    starttime=pool1.index[i]-relativedelta(months=12)  #此处为滚动窗口，可设置滚动窗口参数
    part_data=turnover[starttime:pool1.index[i]]  #截取计算窗口所需要的值
    part_turnover=part_data.apply(condition_turnover)
    return part_turnover

In [82]:
turn_12m=apply_parallel(pool1.index,get_turnover_result)

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    3.2s
[Parallel(n_jobs=7)]: Done 120 out of 120 | elapsed:   10.5s finished


In [83]:
for k in range(len(turn_12m)):
    pool1.iloc[k]=turn_12m[k]
pool1

,S.CN.SSE.600000,S.CN.SSE.600004,S.CN.SSE.600006,S.CN.SSE.600007,S.CN.SSE.600008,S.CN.SSE.600009,S.CN.SSE.600010,S.CN.SSE.600011,S.CN.SSE.600012,S.CN.SSE.600015,...,S.CN.SZSE.300805,S.CN.SZSE.300806,S.CN.SZSE.300807,S.CN.SZSE.300808,S.CN.SZSE.300809,S.CN.SZSE.300810,S.CN.SZSE.300811,S.CN.SZSE.300812,S.CN.SZSE.300813,S.CN.SZSE.300815
Clock,,,,,,,,,,,,,,,,,,,,,
2011-01-31,0.006898,0.009311,0.006912,0.003017,0.008751,0.017355,0.013787,0.00283,0.008538,0.007544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-02-28,0.006965,0.008893,0.007012,0.002989,0.007211,0.017321,0.014238,0.002934,0.006973,0.007781,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-03-31,0.007141,0.008824,0.006883,0.002955,0.00686,0.016879,0.019882,0.00309,0.006354,0.008196,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-04-30,0.007203,0.008671,0.006337,0.003149,0.006591,0.016723,0.026311,0.003315,0.005991,0.008596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-05-31,0.006968,0.008256,0.005987,0.003149,0.006327,0.015718,0.030161,0.003349,0.005915,0.008135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,0.001415,0.009309,0.008583,0.002423,0.004727,0.011848,0.007677,0.00266,0.002958,0.001765,...,0.158618,0.109148,0.100613,0.090195,0.065307,0.093486,0.089476,NaN,NaN,NaN
2020-09-30,0.001455,0.009606,0.009233,0.002574,0.004891,0.011991,0.008107,0.002702,0.002946,0.001786,...,0.155056,0.102335,0.099198,0.089681,0.065959,0.090057,0.082722,0.121414,0.127192,0.07263
2020-10-31,0.001428,0.009538,0.010735,0.002641,0.005019,0.011759,0.008253,0.002732,0.003011,0.001794,...,0.162789,0.097984,0.107736,0.092928,0.067782,0.092388,0.082788,0.118054,0.126326,0.068973


In [84]:
pool1.to_csv("D:/turn_12m.csv")

In [ ]:
#市值因子

In [85]:
cap=valuation_data.pivot_table(index=["Date"],columns=["Symbol"],values="MarketCap")
log_cap=np.log(cap)

In [88]:
log_cap=log_cap.groupby([log_cap.index.year,log_cap.index.month]).tail(1)

In [89]:
log_cap.to_csv("D:/log_cap.csv")